In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2022-08-26T12:51:00.631288-04:00

Python implementation: CPython
Python version       : 3.8.10
IPython version      : 7.28.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.11.0-37-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 32
Architecture: 64bit



In [3]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
import logging

In [4]:
%watermark --iversions

numpy  : 1.22.4
dask   : 2022.7.1
optuna : 2.10.1
pandas : 1.4.3
logging: 0.5.1.2
xgboost: 1.6.2



In [5]:
!nvidia-smi -L

GPU 0: NVIDIA TITAN RTX (UUID: GPU-0eb32f58-b8d5-17c0-e952-f4ec26a9353f)
GPU 1: NVIDIA TITAN RTX (UUID: GPU-50aeb092-88f5-4e0b-7f73-32741666f319)


In [6]:
cluster = LocalCUDACluster(n_workers=2)
client = Client(cluster)

2022-08-26 12:51:13,162 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-08-26 12:51:13,162 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-08-26 12:51:13,196 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-08-26 12:51:13,196 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [7]:
!nvidia-smi

Fri Aug 26 12:51:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:09:00.0 Off |                  N/A |
| 41%   47C    P2    67W / 280W |    167MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    On   | 00000000:41:00.0  On |                  N/A |
| 40%   

In [8]:
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold_d = delayed(pd.read_csv)(f'../input/xgtrain_fold_{i}.csv.gz')
    train_fold = dd.from_delayed(train_fold_d)
    
    val_fold_d = delayed(pd.read_csv)(f'../input/xgval_fold_{i}.csv.gz')
    val_fold = dd.from_delayed(val_fold_d)
    
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4


In [9]:
train = pd.read_csv('../input/train.csv.gz')

target = train['label'].values
target

array([5, 0, 4, ..., 5, 6, 8])

In [10]:
train_oof = np.zeros((target.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        'objective':'multi:softmax', 
        'num_class':10,
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),
        'random_state': 137,

    }
    
    #start_time = time()
    kf = StratifiedKFold(5, shuffle=True, random_state=1974)

    for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i], train_ys[i])
        dval = xgb.dask.DaskDMatrix(client, val_folds[i], val_ys[i])
        
        output = xgb.dask.train(client, params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = predictions
        del dtrain, dval, output
        gc.collect()
        gc.collect()


    acc = accuracy_score(target, train_oof)
    
    return acc

In [11]:
train_oof.shape

(60000,)

In [12]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_0.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(storage="sqlite:///xgb_optuna_mnist_0.db", study_name="five_fold_optuna_xgb_0", direction='maximize')

In [13]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=3)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[12:52:34] task [xgboost.dask-0]:tcp://127.0.0.1:38383 got new rank 0
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[12:54:09] task [xgboost.dask-1]:tcp://127.0.0.1:44829 got new rank 0
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[12:55:41] task [xgboost.dask-0]:tcp://127.0.0.1:38383 got new rank 0
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[12:57:14] task [xgboost.dask-1]:tcp://127.0.0.1:44829 got new rank 0
/home/tunguz

CPU times: user 1min 7s, sys: 20 s, total: 1min 27s
Wall time: 34min 36s


In [14]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,params_tree_method,state
0,0,0.955667,0.002232,0.832674,logloss,1.600160,0.015623,7,125,0.881342,gpu_hist,COMPLETE
1,1,0.917150,1.548932,0.713300,logloss,3.729454,0.007025,20,213,0.564231,gpu_hist,COMPLETE
2,2,0.974667,0.009347,0.788640,logloss,0.989694,0.046231,13,24,0.655625,gpu_hist,COMPLETE


In [15]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_0.csv', index=False)
df.head()

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[13:27:10] task [xgboost.dask-1]:tcp://127.0.0.1:44829 got new rank 0
2022-08-26 13:28:18,264 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
2022-08-26 13:28:18,414 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[13:28:20] task [xgboost.dask-0]:tcp://127.0.0.1:38383 got new rank 0
2022-08-26 13:29:28,357 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
2022-08-26 13:29:28,510 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (th

CPU times: user 1min 11s, sys: 21.4 s, total: 1min 32s
Wall time: 32min 54s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,params_tree_method,state
0,0,0.955667,0.002232,0.832674,logloss,1.600160,0.015623,7,125,0.881342,gpu_hist,COMPLETE
1,1,0.917150,1.548932,0.713300,logloss,3.729454,0.007025,20,213,0.564231,gpu_hist,COMPLETE
2,2,0.974667,0.009347,0.788640,logloss,0.989694,0.046231,13,24,0.655625,gpu_hist,COMPLETE
3,3,0.907283,0.001429,0.954636,logloss,0.003314,0.010473,13,257,0.406071,gpu_hist,COMPLETE
4,4,0.930967,0.159481,0.582950,logloss,0.068239,0.017247,9,217,0.526333,gpu_hist,COMPLETE


In [16]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_0.csv', index=False)
df.head()

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[14:00:05] task [xgboost.dask-0]:tcp://127.0.0.1:38383 got new rank 0
2022-08-26 14:00:59,365 - distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (threshold: 10%)
2022-08-26 14:00:59,523 - distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (threshold: 10%)
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[14:01:01] task [xgboost.dask-1]:tcp://127.0.0.1:44829 got new rank 0
2022-08-26 14:01:55,047 - distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (threshold: 10%)
2022-08-26 14:01:55,206 - distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (th

CPU times: user 2min 52s, sys: 1min 4s, total: 3min 57s
Wall time: 1h 37min 50s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,params_tree_method,state
0,0,0.955667,0.002232,0.832674,logloss,1.600160,0.015623,7,125,0.881342,gpu_hist,COMPLETE
1,1,0.917150,1.548932,0.713300,logloss,3.729454,0.007025,20,213,0.564231,gpu_hist,COMPLETE
2,2,0.974667,0.009347,0.788640,logloss,0.989694,0.046231,13,24,0.655625,gpu_hist,COMPLETE
3,3,0.907283,0.001429,0.954636,logloss,0.003314,0.010473,13,257,0.406071,gpu_hist,COMPLETE
4,4,0.930967,0.159481,0.582950,logloss,0.068239,0.017247,9,217,0.526333,gpu_hist,COMPLETE


/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3468: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3468: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3468: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3468: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3468: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/